In [2]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from util.math import *

In [3]:
uso = yf.download("USO", start="2006-05-24", end="2012-04-09", progress=False)[
    "Adj Close"
].to_numpy()
gld = yf.download("GLD", start="2006-05-24", end="2012-04-09", progress=False)[
    "Adj Close"
].to_numpy()

In [4]:
lookback = 20
hedge_ratio = np.empty(len(uso))
hedge_ratio[:lookback] = np.nan

In [5]:
def get_hedge_ratio(x, y):
    _x = np.vstack((x, np.ones(len(x)))).T
    regression_result = np.linalg.lstsq(_x, y, rcond=None)
    return regression_result[0][0]


for t in range(lookback, hedge_ratio.shape[0]):
    x = gld[t - lookback : t]
    y = uso[t - lookback : t]
    hedge_ratio[t] = get_hedge_ratio(x, y)


In [6]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.arange(len(hedge_ratio)), y=hedge_ratio, mode="lines", name="lines")
)
fig.update_layout(title=dict(text="Hedge Ratio"))


In [7]:
price_mat = np.column_stack((gld, uso))
weights = np.column_stack((-hedge_ratio, np.ones(len(hedge_ratio))))
portfolio = np.sum(price_mat * weights, axis=1)

In [8]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.arange(len(portfolio)), y=portfolio, mode="lines", name="lines")
)
fig.update_layout(title=dict(text="Constructed Portfolio Market Value"))


In [9]:
portfolio = portfolio[lookback:]
hedge_ratio = hedge_ratio[lookback:]
price_mat = price_mat[lookback:,]

In [14]:
entry_z = 1
exit_z = 0
rolling_mean = calculate_rolling_mean(portfolio, 20)
rolling_std = calculate_rolling_std(portfolio, 20)
rolling_z = (portfolio - rolling_mean) / rolling_std

In [15]:
long_entry = rolling_z < -entry_z
long_exit = rolling_z > -exit_z

short_entry = rolling_z > entry_z
short_exit = rolling_z < exit_z

In [26]:
n_unit_long = np.zeros(len(portfolio))
n_unit_short = np.zeros(len(portfolio))

n_unit_long[long_entry] = 1
n_unit_long[long_exit] = 0
n_unit_long = np.where(np.isnan(n_unit_long), 0, n_unit_long)  

n_unit_short[short_entry] = -1
n_unit_short[short_exit] = 0
n_unit_short = np.where(np.isnan(n_unit_short), 0, n_unit_short) 

n_unit_long[0] = 0
n_unit_short[0] = 0

In [27]:
n_units = n_unit_long + n_unit_short

In [28]:
positions = np.tile(n_units[:, np.newaxis], (1, price_mat.shape[1])) * np.column_stack(
    (-hedge_ratio, np.ones(len(hedge_ratio)))
)

In [29]:
pnl = np.sum(shift(positions, 1) * calculate_daily_return(price_mat), axis=1)

In [30]:
ret = pnl / np.sum(np.abs(shift(positions, 1)), axis=1)
ret[np.isnan(ret)] = 0

C:\Users\nileb\AppData\Local\Temp\ipykernel_17176\1041607093.py:1: RuntimeWarning:

invalid value encountered in divide



In [31]:
apr = np.prod(1 + ret) ** (252 / len(ret)) - 1
sharpe = np.sqrt(252) * np.mean(ret) / np.std(ret)

In [32]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(len(ret)), y=np.cumprod(1 + ret) - 1, mode="lines", name="lines"
    )
)
fig.update_layout(title=dict(text="Cumulative return"))


In [33]:
print(f"APR={apr} Sharpe={sharpe}")

APR=0.06710877385085179 Sharpe=0.5296765602134662
